# NFL Models

## Data Export
```
DB_FILE=${FANTASY_HOME}/nfl_hist_2009-2022.scored.db
SEASONS='2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022'
TEAM_STATS='def_fumble_recov def_int def_sacks def_tds op_* passing_yds pen_yds pens pts rushing_yds turnovers win'

# WR TE
dumpdata.sc ${DB_FILE} \
    --seasons $SEASONS --pos WR TE --no_teams \
    --stats tds "receiving*" \
    --target_calc_stats "*_score_off" --current_extra venue is_home \
    --player_team_stats $TEAM_STATS --opp_team_stats $TEAM_STATS \
    --hist_recent_games 3 --hist_recent_mode ma \
    --progress -f nfl_WRTE.csv

# RB
dumpdata.sc ${DB_FILE} \
    --seasons $SEASONS --pos RB --no_teams \
    --stats tds "receiving*" "rushing*" \
    --current_extra venue is_home --target_calc_stats "*_score_off" \
    --player_team_stats $TEAM_STATS --opp_team_stats $TEAM_STATS \
    --hist_recent_games 3 --hist_recent_mode ma \
    --progress -f nfl_RB.csv

# QB
dumpdata.sc ${DB_FILE} \
    --seasons $SEASONS --no_teams --pos QB \
    --stats tds "rushing*" "passing*" \
    --target_calc_stats "*_score_off" --current_extra venue is_home \
    --player_team_stats $TEAM_STATS --opp_team_stats $TEAM_STATS \
    --hist_recent_mode ma --hist_recent_games 3 \
    --progress -f nfl_QB.csv

# team defence and win
dumpdata.sc ${DB_FILE} \
    --seasons $SEASONS --no_players \
    --stats $TEAM_STATS --opp_team_stats $TEAM_STATS \
    --current_extra venue is_home \
    --target_calc_stats "*_score_def" --target_stat pts win \
    --hist_recent_games 3 --hist_recent_mode ma \
    --progress -f nfl_team.csv
```

In [ ]:
RANDOM_SEED = 1
TRAINING_TIME = 600
VALIDATION_SEASON = 2022
RECENT_GAMES = 3
TRAINING_SEASONS = [
    2009,
    2010,
    2011,
    2012,
    2013,
    2014,
    2015,
    2016,
    2017,
    2018,
    2019,
    2020,
    2021,
]
REUSE_EXISTING = False
MISSING_DATA_THRESHOLD = .07

In [ ]:
import sys

sys.path.append("..")

from fantasy_py import PlayerOrTeam
from train_test import load_data, model_and_test

TARGET = ("stat", "pts")
MODEL_NAME = "NFL-team-pts"
DATA_FILENAME = "nfl_team.csv"
P_OR_T = PlayerOrTeam.TEAM
INCLUDE_POS = None
TARGET_POS = None

# TARGET = ("stat", "win")
# MODEL_NAME = "NFL-team-win"
# DATA_FILENAME = "nfl_team.csv"
# P_OR_T = PlayerOrTeam.TEAM
# INCLUDE_POS = None
# TARGET_POS = None

# TARGET = ("calc", "dk_score_def")
# MODEL_NAME = "NFL-DEF-DK"
# DATA_FILENAME = "nfl_team.csv"
# P_OR_T = PlayerOrTeam.TEAM
# INCLUDE_POS = None
# TARGET_POS = None

# TARGET = ("calc", "dk_score_off")
# MODEL_NAME = "NFL-QB-DK"
# DATA_FILENAME = "nfl_QB.csv"
# P_OR_T = PlayerOrTeam.PLAYER
# INCLUDE_POS = False
# TARGET_POS = ["QB"]

# TARGET = ("calc", "dk_score_off")
# MODEL_NAME = "NFL-RB-DK"
# DATA_FILENAME = "nfl_RB.csv"
# P_OR_T = PlayerOrTeam.PLAYER
# INCLUDE_POS = False
# TARGET_POS = ["RB"]

# TARGET = ("calc", "dk_score_off")
# MODEL_NAME = "NFL-WRTE-DK"
# DATA_FILENAME = "nfl_WRTE.csv"
# P_OR_T = PlayerOrTeam.PLAYER
# INCLUDE_POS = True
# TARGET_POS = ["WR", "TE"]

raw_df, tt_data, one_hot_stats = load_data(
    DATA_FILENAME,
    TARGET,
    VALIDATION_SEASON,
    include_position=INCLUDE_POS,
    seed=RANDOM_SEED,
)

assert list(one_hot_stats.keys()) == ["extra:venue"]


for automl_type in ["tpot"]:  # , "autosk"]:
    model = model_and_test(
        MODEL_NAME,
        VALIDATION_SEASON,
        tt_data,
        TARGET,
        TRAINING_TIME,
        automl_type,
        P_OR_T,
        RECENT_GAMES,
        TRAINING_SEASONS,
        seed=RANDOM_SEED,
        target_pos=TARGET_POS,
        training_pos=TARGET_POS,
        raw_df=raw_df,
        reuse_existing=REUSE_EXISTING,
    )